In [1]:
import pandas as pd
import numpy as np

In [2]:
qdf = pd.read_csv(r'D:\Green-Gauge\chat\static\csv\query_dataset.csv')

In [3]:
qdf

,query,tag
0,Hii,greet
1,Hello,greet
2,what is green gauge,greengauge
3,tell me about green gauge?,greengauge
4,what is in the blog section,blog
5,what type of blogs are there,blog
6,what type of articles are there?,blog
7,what are the tasks,task
8,what type of tasks are there,task
9,what is task,task


In [4]:
X  = qdf['query'].tolist()
X

['Hii',
 'Hello',
 'what is green gauge',
 'tell me about green gauge?',
 'what is in the blog section',
 'what type of blogs are there',
 'what type of articles are there?',
 'what are the tasks',
 'what type of tasks are there',
 'what is task',
 'what type of merchandise you offer?',
 'what do you offer in merchandise section',
 'what kind of things you sell',
 'how do i get my report',
 'how is the report made?',
 'what is the purpose of green gauge?']

In [5]:
y = qdf['tag'].tolist()
y

['greet',
 'greet',
 'greengauge',
 'greengauge',
 'blog',
 'blog',
 'blog',
 'task',
 'task',
 'task',
 'merchandise',
 'merchandise',
 'merchandise',
 'report',
 'report',
 'greengauge']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer( stop_words='english', max_df=0.7)
X_vectorized = vectorizer.fit_transform(X)
X_vectorized

<16x19 sparse matrix of type '<class 'numpy.float64'>'
	with 31 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(y)
y

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

In [8]:
print(lb.classes_)

['blog' 'greengauge' 'greet' 'merchandise' 'report' 'task']


In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
model = RandomForestClassifier(random_state=1)
model.fit(X_vectorized,y) # algorithm with learn to identify tags

RandomForestClassifier(random_state=1)

In [11]:
# testing
text  = 'how far can you deliver for free?'
input_data = vectorizer.transform([text])
print(input_data.toarray())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
result = model.predict(input_data) # answer

In [13]:
lb.inverse_transform(result)

array(['blog'], dtype='<U11')

In [14]:
# testing
text  = 'blog?'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['blog']


In [15]:
# testing
text  = 'what type of merchandise you offer?'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['merchandise']


In [16]:
text =" how is the report made?	"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['report']


In [17]:
text ="tell me about green gauge?"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['greengauge']


In [18]:
text ="what type of tasks are there"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['task']


In [19]:
text= 'what is task'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
ai_tag = lb.inverse_transform(result)[0]

# getting bot reply

In [20]:
rdf = pd.read_csv(r'D:\Green-Gauge\chat\static\csv\bot_response_dataset.csv')

In [21]:
resultdf = rdf[rdf['tag']==ai_tag]

In [22]:
resultdf.head()

,response,tag
6,These tasks help you to attain a more sustaina...,task
7,Complete the tasks to have a more sustainable ...,task


In [23]:
responses = resultdf.response.to_list()

In [24]:
import random

In [25]:
random.choice(responses)

'These tasks help you to attain a more sustainable life style'

In [26]:
import pickle

In [27]:
# classes
# binarzer
# model 
# vectorizer
model_data = {
    'binarizer': lb,
    'model' : model,
    'vectorizer':vectorizer,
    'classes':lb.classes_.tolist()
}

In [28]:
with open('model.pkl','wb') as f:
    pickle.dump(model_data, f)

# testing saved model

In [29]:
model_loaded_data = {}

with open('model.pkl','rb') as f:
    model_loaded_data = pickle.loads(f.read())

In [30]:
def predict_tag(txt='Hello world', vectorizer=None,
                model=None,binarizer=None,*args,**kwargs):
    input_vector = vectorizer.transform([txt])
    result = model.predict(input_vector)
    output_tag = lb.inverse_transform(result)
    return output_tag[0]

In [31]:
def get_bot_reply(predicted_tag):
    resultdf = rdf[rdf['tag']==predicted_tag] 
    responses = resultdf.response.to_list()
    return random.choice(responses)

In [32]:
text ="what type of merchandise you offer?"
tag = predict_tag(text,**model_loaded_data)
get_bot_reply(tag)

'We offer merchandise made of recycled and sustainable material'

In [33]:
text= 'what is task'
tag = predict_tag(text,**model_loaded_data)
get_bot_reply(tag)

'Complete the tasks to have a more sustainable life'

In [34]:
tag = predict_tag(qdf.iloc[5].query, **model_loaded_data)
get_bot_reply(tag)

'We have blogs related to sustainable living practices '

In [35]:
for query in qdf['query'].to_list():
    tag = predict_tag(query, **model_loaded_data)
    print(get_bot_reply(tag))

Hello and welcome
Hii and welcome to Green Gauge
Green gauge is an integrated platform where you can complete task , read blogs and buy merchandise
Green Gauge is a platform to make people aware about sustainability living practices
We have blogs related to sustainable living practices 
We have blogs related to sustainable living practices 
We have blogs related to sustainable living practices 
Complete the tasks to have a more sustainable life
These tasks help you to attain a more sustainable life style
These tasks help you to attain a more sustainable life style
We offer a wide range of products like bamboo bottles,shoes,etc
We offer merchandise made of recycled and sustainable material
We offer merchandise made of recycled and sustainable material
Complete more tasks to get more points
Complete more tasks to get more points
Green gauge is an integrated platform where you can complete task , read blogs and buy merchandise
